In [1]:
%load_ext autoreload
%load_ext tensorboard
%autoreload 2
import os
import sys

import clip

import importlib

import contextlib

import json
import os
import random


sys.path.insert(0, os.path.abspath('../..')) 
sys.path.insert(0, os.path.abspath('../../..'))

sys.path.insert(0, os.path.abspath('../../../../stablediffusion'))


In [2]:

import numpy as np
import torch 
from collections import defaultdict
from utils import get_default_path
from utils import Stopwatch
import time
import datetime

In [23]:
from open_clip.tokenizer import _tokenizer as clip_tokenizer
clip_tokenizer.decoder[len(clip_tokenizer.decoder)-1]

'<end_of_text>'

In [3]:



import ai.stabledisco as sd
import ai.torchmodules as torchmodules
import ai.torchmodules.data as torchdata
import ai.torchmodules.utils as torchutils
import ai.stabledisco.utils as sdutils
import ai.stabledisco.data as sddata
import ai.stabledisco.constants as sdconsts
import ai.torchmodules.pruning as torchprune
import torch
import torch.nn as nn
import pandas as pd

import ai.stabledisco.decoderpipeline as decoderpipeline
  
tmp_dir =  "/home/ubuntu/tmp/"
tokens_path = os.path.join(tmp_dir, "text_tokens.pk")
rev_tokens_path = os.path.join(tmp_dir, "rev_text_tokens.pk")

In [4]:
tmp_dir =  "/home/ubuntu/tmp/"
features_path = os.path.join(tmp_dir, "text_features_curr.pk")
tokens_path = os.path.join(tmp_dir, "text_tokens_curr.pk")
rev_tokens_path = os.path.join(tmp_dir, "rev_text_tokens_curr.pk")

features_path_old = os.path.join(tmp_dir, "text_features_old.pk")
tokens_path_old = os.path.join(tmp_dir, "text_tokens_old.pk")
rev_tokens_path_old = os.path.join(tmp_dir, "rev_text_tokens_old.pk")

curr_toks = torch.load(tokens_path)
#old_toks = torch.load(tokens_path_old)

In [4]:
if len(text_tokens.shape) > 1:
    text_tokens.view(1, text_tokens.size(0))
flipped = torch.flip(text_tokens, dims=(1,))
end_idx_arg = torch.argwhere(flipped == sdconsts.eot_token)[:,1]
for idx in range(flipped.size(0)):
    flipped[idx] = torch.cat((flipped[idx,end_idx_arg[idx]:], flipped[idx,:end_idx_arg[idx]]))


text_tokens = torch.save(flipped, rev_tokens_path)

In [7]:
import open_clip
tokenizer = open_clip.get_tokenizer('ViT-H-14')


In [5]:
import open_clip
vit14_clip_model, _,  vit14_clip_preprocess = open_clip.create_model_and_transforms('ViT-H-14', device=torch.device('cuda'), pretrained='laion2b_s32b_b79k')
vit14_clip_model = vit14_clip_model.cuda()

vit14_clip_model = vit14_clip_model.eval()
for param in vit14_clip_model.parameters():
    param.requires_grad = False
    
clip_model = sd.ClipModel(vit14_clip_model, vit14_clip_preprocess, "ViT-H-14").float()

In [6]:
feature_stack = []
step = 25000
with torch.no_grad():
    for start in range(0, len(curr_toks), step):

        print(start)
        new_features = clip_model.get_features(curr_toks[start:start+step].cuda())
        feature_stack.append(new_features.cpu())
        del new_features

    features = torch.stack(tuple(feature_stack))

0
25000
50000
75000
100000
125000
150000
175000
200000
225000
250000
275000
300000
325000
350000
375000
400000
425000
450000
475000
500000
525000
550000
575000
600000
625000
650000
675000
700000
725000
750000
775000
800000
825000
850000
875000
900000
925000
950000
975000
1000000
1025000
1050000
1075000
1100000
1125000
1150000
1175000
1200000
1225000
1250000
1275000
1300000
1325000
1350000
1375000
1400000
1425000
1450000
1475000
1500000
1525000
1550000
1575000
1600000
1625000
1650000
1675000
1700000
1725000
1750000
1775000
1800000
1825000
1850000
1875000
1900000
1925000
1950000
1975000
2000000
2025000
2050000
2075000
2100000
2125000
2150000
2175000
2200000
2225000
2250000
2275000
2300000
2325000
2350000
2375000
2400000
2425000
2450000
2475000
2500000
2525000
2550000
2575000
2600000
2625000
2650000
2675000
2700000
2725000
2750000
2775000
2800000
2825000
2850000
2875000
2900000
2925000
2950000
2975000
3000000
3025000
3050000
3075000
3100000
3125000
3150000
3175000
3200000
3225000
3250000


In [8]:
features = torch.cat(feature_stack, axis=0)

In [10]:
del feature_stack

In [11]:
torch.save(features, features_path)

In [ ]:
feature_stack = []
step = 10000
with torch.no_grad():
    for start in range(0, len(old_toks), step):
        print(start)
        new_features = clip_model.get_features(old_toks[start:start+step].cuda())
        feature_stack.append(new_features.cpu())

    features = torch.stack(tuple(feature_stack))
    torch.save(features, features_path_old)